In [31]:
# import packages
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

In [2]:
import pyodbc
# Trusted Connection to Named Instance
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=PADKBASVR02\SQL2012;DATABASE=DW_Vector;Trusted_Connection=yes;')

In [3]:
df = pd.read_sql_query("with z as (SELECT [TrapDate],[Latitude],[Longitude],[LocationDescription] AS 'Location',[Site], sum(case when SpeciesDescription = 'Mosquito' then VectorCount end) 'MosquitoCount', sum(case when SpeciesDescription = 'Midges' then VectorCount end) 'MidgeCount' FROM [DW_Vector].[Vector].[VectorCount]	WHERE [VectorCount] IS NOT NULL AND [LocationDescription] NOT LIKE 'off%' AND [TrapDate] >= '2022-11-01' AND [IsActive] = 1 group by [TrapDate],[IsActive],[Latitude],[Longitude],[LocationDescription],[Site])select *, sum(MosquitoCount) over(partition by location order by trapdate) 'MosquitoCummulative', sum(MidgeCount) over(partition by location order by trapdate) 'MidgeCummulative' from z order by TrapDate",conn)
#df

In [4]:
df.head()

,TrapDate,Latitude,Longitude,Location,Site,MosquitoCount,MidgeCount,MosquitoCummulative,MidgeCummulative
0,2022-11-01,-16.542175,28.871361,Btween Breeder Pen 1 &2,NCF,1,18,1,18
1,2022-11-01,-16.543731,28.872550,Btwn F10/11,KCF,0,17,0,17
2,2022-11-01,-16.543131,28.874519,Btwn F3/Hatch,KCF,0,2,0,2
3,2022-11-01,-16.539750,28.877203,Dumpsite,NCF,0,0,0,0
4,2022-11-01,-16.543017,28.871311,F14 Western side,KCF,0,4,0,4


In [6]:
mapbox_access_token =  'pk.eyJ1Ijoia2h1bWJ1bGFuaWMiLCJhIjoiY2xhcjBobDloMW9mbDNvdGFxYXhhY3l1YSJ9.Te1bDqF8QI-pFYQGcXmQUQ'

In [33]:
px.set_mapbox_access_token(mapbox_access_token)
fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude', 
                        #animation_frame = 'TrapDate',
                        #animation_group='Location',
                        color='MosquitoCount', 
                        size='MosquitoCount', 
                        size_max=30, 
                        #category_orders={'TrapDate':list(df.TrapDate.unique())},
                        color_discrete_sequence=['#008000', 'orange', '#FF0000'],
                        #range_color=(0,5),
                        #color_continuous_midpoint='VectorCount',
                         #hover_data=1,
                        hover_name='Location',
                        zoom=7,
                        #center=dict(lon=105, lat=38),
                        width=800, 
                        height=600,
                        mapbox_style='mapbox://styles/khumbulanic/clascopvx002f14mf1pq6vbps'
                        #mapbox_style='carto-positron'
                       )
fig.update_layout(
    mapbox_style='mapbox://styles/khumbulanic/clascopvx002f14mf1pq6vbps')
#Adjust pitch and bearing to adjust the rotation
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, 
                  mapbox=dict(
                      pitch=60,
                      bearing=30,
                      zoom=15,
                  ))

fig.show()
#fig.show(renderer="browser")

In [30]:
len(df['TrapDate'].unique())

26

In [13]:
import plotly.express as px
px.set_mapbox_access_token(mapbox_access_token)

frame = []
for frame in range(len(df['TrapDate'].unique())):
    x_axis_frame = np.arange(frame)
    y_axis_frame = list(df.iloc[0,1:frame])
    curr_frame = go.frame(df, [go.scatter(x=x_axis_frame , y=y_axis_frame, mode= 'lines',)])
    frames.append(curr_frame)

fig = px.scatter_mapbox(df, 
                        lat="Latitude", 
                        lon="Longitude",     
                        color="MosquitoCummulative", 
                        size="MosquitoCummulative",
                        #animation_frame = 'TrapDate',
                        #animation_group='Location',
                        color_discrete_sequence=['#008000', 'orange', '#FF0000'],
                        hover_name='Location',
                        size_max=50, 
                        zoom=10,
                        mapbox_style='mapbox://styles/khumbulanic/clascopvx002f14mf1pq6vbps')
fig.show()

In [23]:
import plotly.graph_objects as go
px.set_mapbox_access_token(mapbox_access_token)
fig = go.Figure(go.Scattermapbox(
                                    lat=df.Longitude,
                                    lon=df.Longitude,
                                    mode='markers',
    
                                   # mapbox_style='mapbox://styles/khumbulanic/clascopvx002f14mf1pq6vbps'
                                    #marker=go.scattermapbox.marker(size=df.MosquitoCount)
                                )
               )

fig.show()

In [27]:
import plotly.express as px
#df = px.data.gapminder()
px.scatter(df, x="MidgeCummulative", y="MosquitoCummulative", animation_frame="TrapDate", animation_group="Location",
           size="MosquitoCummulative", color="Location", hover_name="Location",
           category_orders={'TrapDate':list(df.TrapDate.unique())},
           size_max=50, range_x=[0,7000], range_y=[0,250])